In [1]:
# Required Libraries
import os
import math
import numpy as np
from skimage import io  # For reading images
import cv2  # OpenCV for image processing
from sklearn.model_selection import train_test_split  # Splitting data
from sklearn.neighbors import KNeighborsClassifier  # k-NN Classifier
from sklearn.metrics import accuracy_score  # For accuracy evaluation

# Dataset Path
dataset_path1 = '../templates'
letters_data1 = []  # Feature vectors
labels1 = []  # Labels (folder names)
minHeight1 = math.inf  # Initialize with infinite height
minWidth1 = math.inf  # Initialize with infinite width

# Find minimum dimensions for resizing
for letter_folder1 in os.listdir(dataset_path1):
    letter_path1 = os.path.join(dataset_path1, letter_folder1)
    if os.path.isdir(letter_path1):  # Ensure it's a directory
        for filename in os.listdir(letter_path1):
            if filename.endswith('.png'):  # Process PNG files
                image_path = os.path.join(letter_path1, filename)
                img = io.imread(image_path)
                if img.shape[0] < minHeight1:
                    minHeight1 = img.shape[0]
                if img.shape[1] < minWidth1:
                    minWidth1 = img.shape[1]

# Define uniform dimensions for all images
new_dimensions1 = (minWidth1, minHeight1)

# Preprocess images and extract features
for letter_folder1 in os.listdir(dataset_path1):
    letter_path1 = os.path.join(dataset_path1, letter_folder1)
    if os.path.isdir(letter_path1):
        for filename in os.listdir(letter_path1):
            if filename.endswith('.png'):
                image_path1 = os.path.join(letter_path1, filename)
                original_image1 = io.imread(image_path1)
                img1 = cv2.resize(original_image1, new_dimensions1)  # Resize image

                # Handle RGBA to RGB conversion
                if len(img1.shape) == 3 and img1.shape[2] == 4:
                    img1 = cv2.cvtColor(img1, cv2.COLOR_RGBA2RGB)

                # Convert to grayscale
                img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

                # Flatten the image into a feature vector
                flattened_image1 = img1.flatten()
                letters_data1.append(flattened_image1)
                labels1.append(letter_folder1)  # Folder name as label

# Convert lists to NumPy arrays
X1 = np.array(letters_data1)
y1 = np.array(labels1)

# Split the dataset into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.01, random_state=0)

# Initialize and train the k-NN classifier
knn_classifier1 = KNeighborsClassifier(n_neighbors=3)
knn_classifier1.fit(X_train1, y_train1)

# Make predictions on the test set
y_pred1 = knn_classifier1.predict(X_test1)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test1, y_pred1)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


In [ ]:
# Function to preprocess a single image
def preprocess_image(image_path, dimensions):
    """
    Preprocess the input image to match the training format.
    """
    # Load the image
    img = io.imread(image_path)
    
    # Resize to the new dimensions
    img_resized = cv2.resize(img, dimensions)
    
    # Convert RGBA to RGB if necessary
    if len(img_resized.shape) == 3 and img_resized.shape[2] == 4:
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_RGBA2RGB)
    
    # Convert to grayscale
    img_gray = cv2.cvtColor(img_resized, cv2.COLOR_RGB2GRAY)
    
    # Flatten the image
    flattened_img = img_gray.flatten()
    
    return flattened_img

# Path to your test image
test_image_path = 'path/to/your/test_image.png'

# Preprocess the test image
preprocessed_image = preprocess_image(test_image_path, new_dimensions1)

# Reshape to 2D array (required for sklearn input)
preprocessed_image = np.array(preprocessed_image).reshape(1, -1)

# Make prediction using the trained k-NN classifier
predicted_label = knn_classifier1.predict(preprocessed_image)

# Display the result
print(f"Predicted Label: {predicted_label[0]}")
